In [1]:
import Pkg
Pkg.activate("../../../FinancialPlanner")

  Activating project at `c:\Users\matsz\programowanie\Optymalizacja_portfela\FinancialPlanner`


In [2]:
using Revise
using Turing
using LinearAlgebra
using Distributions
using Random
using StatsPlots
using PDMats
using StatsBase
using Base.Iterators
using DataFrames, TimeSeries, XLSX
using PrettyTables

In [3]:
include("../VARs/utils.jl")
includet("../RS/hmm_est.jl")

In [6]:
df = DataFrame(XLSX.readtable("../../data/usa_data.xlsx", "Data", infer_eltypes=true))
data_source = collapse(TimeArray(df; timestamp = :Date), Dates.month, last)

freq = 12

macro_data = percentchange(data_source[:CPI, :INDPRO], :log)

start = Date(1955, 01,01)
end_d = Date(2024, 07, 28)



display("Macro data")
display(returns_summarystats(to(from(macro_data, start),end_d), freq))

display("Correlations")
display(cor(values(to(from(macro_data, start),end_d)))) 

"Macro data"

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
CPI,0.0123,0.2148,0.0354,0.0596,-0.2321,0.0851,0.0035,0.5472,2.7577,2.8885,0.0348
INDPRO,0.0358,0.7653,0.0243,0.0788,-1.7045,-3.1785,-0.0232,0.3076,48.6631,0.6803,0.0294


nothing

"Correlations"

2×2 Matrix{Float64}:
  1.0       -0.011246
 -0.011246   1.0

In [7]:
assets_names = [:CPI, :INDPRO]
n_assets = length(assets_names)

2

In [16]:
n_regimes = 4
hmm = hmm_est(macro_data, n_regimes)
regime_summary(hmm[1], assets_names, 12)


Regime,CPI,INDPRO
1,0.009,0.038
2,0.103,0.024
3,0.015,-0.019
4,0.047,0.027


Regime,CPI,INDPRO
1,0.007,0.019
2,0.010,0.024
3,0.015,0.084
4,0.006,0.020


,CPI,INDPRO
CPI,1.000,0.169
INDPRO,0.169,1.000


,CPI,INDPRO
CPI,1.000,-0.149
INDPRO,-0.149,1.000


,CPI,INDPRO
CPI,1.000,-0.094
INDPRO,-0.094,1.000


,CPI,INDPRO
CPI,1.000,0.066
INDPRO,0.066,1.000


Regimes transition matrix


Regime,1,2,3,4
1,0.800,0.000,0.024,0.176
2,0.013,0.851,0.062,0.075
3,0.103,0.043,0.854,0.000
4,0.183,0.032,0.006,0.780


In [15]:
scenarios = simulate_hmm(hmm[1], n_assets, 25*freq, 10_000)

periods = [1,5, 10, 25]
ret_in_years = cum_returns_in_periods(scenarios, periods, freq, true)
print_scenarios_summary(ret_in_years, assets_names, string.(periods))
n_assets = length(assets_names)
for a in 1:n_assets
    print_scenarios_percentiles(ret_in_years[a, :, :], [.01, .025, .05, .25, .5, .75, .95, .975, .99], string.(periods), string.(assets_names[a]))
end  

,CPI,INDPRO
1,0.0268,0.0047
5,0.0326,0.0202
10,0.0334,0.0227
25,0.0338,0.024


,CPI,INDPRO
1,0.0223,0.0647
5,0.0124,0.021
10,0.009,0.0139
25,0.0057,0.0083


,CPI,INDPRO
1,0.9925,-0.3881
5,0.8585,-0.5034
10,0.6487,-0.4744
25,0.4217,-0.3355


,CPI,INDPRO
1,1.1737,0.7441
5,1.1045,0.8572
10,0.5341,0.7073
25,0.1981,0.2987


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.0124,-0.007,-0.0025,0.0117,0.0231,0.0368,0.0728,0.0845,0.0943
5,0.0108,0.0137,0.016,0.0238,0.0306,0.0396,0.0561,0.0614,0.0683
10,0.0171,0.0189,0.0205,0.0269,0.0323,0.0388,0.05,0.0535,0.0586
25,0.0222,0.024,0.0252,0.0297,0.0333,0.0374,0.0438,0.0461,0.0486


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.1683,-0.1371,-0.1094,-0.0336,0.0096,0.0469,0.1011,0.1231,0.1513
5,-0.0387,-0.0268,-0.0176,0.0083,0.022,0.0343,0.0511,0.0569,0.0649
10,-0.0154,-0.0076,-0.002,0.0145,0.0237,0.0321,0.0433,0.047,0.0519
25,0.0025,0.0064,0.0097,0.0188,0.0245,0.0298,0.0367,0.0388,0.0416


In [81]:
mdd, mddl = max_drawdown_and_length(scenarios[1,:,:])

display(assets_names[1])
display("mdd")
display(quantile(mdd, [.03, .25, .5, .75, .97])')
display("mddl")
display(quantile(mddl, [.03, .25, .5, .75, .97])')

mdd, mddl = max_drawdown_and_length(scenarios[2,:,:])

display(assets_names[2])
display("mdd")
display(quantile(mdd, [.03, .25, .5, .75, .97])')
display("mddl")
display(quantile(mddl, [.03, .25, .5, .75, .97])')

:Total_Stock_CPI

"mdd"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 0.212073  0.312126  0.388067  0.48415  0.673097

"mddl"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 26.0  48.0  70.0  104.0  209.03

:Price_10Y_CPI

"mdd"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 0.138995  0.196629  0.246045  0.309159  0.45166

"mddl"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 42.0  78.0  113.0  168.0  285.0